# This program processes FASTA files for analysis

## Parse FASTA File and Match Taxonomic Data

In [ ]:
import os
import csv
import pandas as pd
from collections import defaultdict
from pathlib import Path
import sqlite3
from Bio import SeqIO

# for debugging
from itertools import islice

### Developing Tree Model "taxo_search_tree" for Fast Taxonomic Lookup

In [ ]:
# Table containing all taxonomic relationships from ICTV
taxonomy_table = pd.read_csv("taxonomy_table.csv", index_col=0)

taxonomy_table

In [ ]:
# extract info from taxonomy_table and build lookup tree

# define tree
taxo_search_tree = defaultdict(str)


# iterate through entire table and define relationships

# itertuples is faster for row iteration than pandas series
for row_tuple in taxonomy_table.itertuples(index=False, name = None):

    
    # Iterate through levels (EXCLUDE SUBLEVELS) in reverse order (known size of row = 15)
    for i in range(14,-2,-2):

        raw_value = row_tuple[i]

        if pd.notna(raw_value):
            current = raw_value.lower()
            
            if taxo_search_tree[current] == "":
                # Point lower level to higher

                next_level = row_tuple[i-2]

                if pd.notna(next_level):
                    taxo_search_tree[current] = next_level.lower()
                else:
                    taxo_search_tree[current] = next_level

    # Iterate through sublevels in reverse order
    for i in range(13,-1,-2):

        
        raw_value = row_tuple[i]

        if pd.notna(raw_value):
            current = raw_value.lower()
            
            if taxo_search_tree[current] == "":
                # Point lower level to related level

                next_level = row_tuple[i-2]

                if pd.notna(next_level):
                    taxo_search_tree[current] = next_level.lower()
                else:
                    taxo_search_tree[current] = next_level

# Completed tree is named: taxo_search_tree

### Parse FASTA files into table

In [ ]:
# Point at folder with fasta files
fasta_dir = "virusDB/NCBI_Protein_Fasta"

# Grab all Fasta files paths

fasta_paths = list(Path(fasta_dir).glob(f'*.{"fsa_aa"}'))

In [ ]:
# To determine if entry in taxo_search_tree

def find_taxo_key(words, taxo_search_tree):
    # Checks all consecutive word combinations in the label against the tree keys
    # Returns the longest matching key found

    N = len(words)
    best_match = None
    
    # 1. Generate Combinations and Search
    # Iterate through all possible starting positions (i)
    for i in range(N):
        # Iterate through all possible ending positions (j)
        # Note: We iterate from the start of the combination (i) to the end (N)
        for j in range(i, N):
            
            # Combine the words from index i to j (inclusive)
            candidate = " ".join(words[i : j + 1])
            
            # 2. O(1) Dictionary Lookup
            if candidate in taxo_search_tree:
                # Update the best match (we prefer longer matches)
                if best_match is None or len(candidate) > len(best_match):
                    best_match = candidate
                    
    return best_match



In [ ]:
valid_files = []

count = 0


# islice(iterator, stop) gets items from the iterator until 'stop' is reached.
for index, path in enumerate(fasta_paths):
    for record in SeqIO.parse(path, "fasta"):
        full_header = record.description

        # extract organism name inside brackets using regex
        match = re.search(r'\[(.+?)\]', full_header)
        organism = match.group(1) if match else ""

        # Find the longest consecutive substring that is in the tree
        taxo_key = find_taxo_key(organism.lower().split(),taxo_search_tree)

    
        # Check if key in the tree
        if taxo_key != None:
            count+=1
        
            
print(f"Count is {count}")

In [ ]:
# Create SQLtable for very large table

conn = sqlite3.connect("bacteria.db")

# Create table scheme with sql
conn.execute("""
CREATE TABLE IF NOT EXISTS sequences (
  protein_id TEXT,
  protein_seq TEXT,
  taxonomic_label TEXT,
  description TEXT
)
""")

In [ ]:
def parse_fasta (path):

    # Scan through every record in fasta
    rows = []
    for record in SeqIO.parse(path, "fasta"):

        # name of the folder containing fasta
        folder_name = os.path.basename(os.path.dirname(path))
        
        full_header = record.description

        # extract organism name inside brackets using regex
        match = re.search(r'\[(.+?)\]', full_header)
        organism = match.group(1) if match else ""
        
        # create substring without bracketed part
        no_id = full_header[len(record.id):].lstrip(" :")        
        pre_description = re.sub(r'\s*\[.+?\]', "", no_id)


        rows.append({
            "genome_id": folder_name,
            "protein_id": record.id,
            "protein_seq": str(record.seq),
            "taxonomic_label": organism,    
            "description": pre_description
        })

    # print (*rows);
    return rows;